# FLUJO DE TRABAJO PARA ESTANDARIZACIÓN DE COLOR Y OBTENCIÓN DE VARIABLES MORFOLÓGICAS Y DE COLOR

## FASE 1.
Obtención de las matrices para crear los archivos npz para usarlos en otras imágenes. También se obtiene el área, largo y ancho de los chips

# DESCRIPTION



In [ ]:
# Import libraries

#import cv2
#import os
#import pandas as pd
#import math
#%matplotlib widget
import numpy as np
from plantcv import plantcv as pcv
#from collections import Counter
from plantcv.parallel import WorkflowInputs
# Set global debug behavior to None (default), "print" (to file), 
# or "plot" (Jupyter Notebooks or X11)

# HSV and CIE-Lab from RGB values
#from ConvRGBtoCIELab import ColorTrans,rgb2lab, rgb_to_hsv 
#from utilities import convert_rgb_to_lab_hsv

In [ ]:
# Input/output options IMG_4245
args = WorkflowInputs(
    #images=["img/color_image.jpg"],    
    images=["./img/colorchart.jpg"],   
    names="image1",
    result="ml_tutorial_results.json",
    outdir="./res",
    writeimg=True,
    debug="plot"
    )

# Set debug to the global parameter 
pcv.params.debug = args.debug

# Change display settings
pcv.params.dpi = 170
pcv.params.text_size = 2
pcv.params.text_thickness = 2



In [ ]:
# Read color checker classic
img_color, path_c,filename_c = pcv.readimage(filename=args.image1)
# If your image is not straight, rotate it. 

rotate_img_c = pcv.transform.rotate(img_color, 90, False)



In [ ]:
# OBTENCIÓN DE LAS MATRICES: card_matrix
card_mask = pcv.transform.detect_color_card(rgb_img=rotate_img_c, adaptive_method=1, block_size=31, radius=40)

headers, card_matrix = pcv.transform.get_color_matrix(rgb_img=rotate_img_c, mask=card_mask)



# Define the standard color card matrix, we know what the colors of those chips should be in an "ideal" image, 
# so we will correct to those values as the TARGET
# Look at where your white chip is in the image to determine which position your card is in (pos)

#pos     = reference value indicating orientation of the color card. The reference
       #         is based on the position of the white chip:
        #        pos = 0: bottom-left corner
        #        pos = 1: bottom-right corner
        #        pos = 2: top-right corner
        #        pos = 3: top-left corner

std_color_matrix = pcv.transform.std_color_matrix(pos=3)

In [ ]:
#Save matrix

fname = "./params/cardcolor_matrix.npz"
c_matrix = np.matrix(card_matrix)
pcv.transform.save_matrix(matrix=c_matrix, filename=fname)

fname2 = "./params/std_color_matrix.npz"
std_matrix=np.matrix(std_color_matrix)
pcv.transform.save_matrix(matrix=std_matrix, filename=fname2)

In [ ]:

#Manually Obtain morphological values 
chip_length=12 #mm

avg_chip_size = pcv.outputs.metadata['median_color_chip_size']['value'][0]
chip_width_px = pcv.outputs.metadata['median_color_chip_width']['value'][0]  # px
chip_height_px = pcv.outputs.metadata['median_color_chip_height']['value'][0]  # px

chip_length_avg=(chip_width_px+chip_height_px)/2 # mm
ratio_length=chip_length/chip_length_avg 

ratio_area=(chip_length**2)/chip_length_avg**2

print(f'chip_length_avg: {chip_length_avg}')
print(f'ratio_length: {ratio_length}')
print(f'ratio_area: {ratio_area}')
